# Gradient Descent

Machine learning applications in research have proven to be of high-value, especially in process development. This reason is that the fruits of well defined ML/deep-learning models largely reflect the needs of our customers: optimization. We have clear outcome goals (cost of ownership, device performance and yield, etc), with robust process data, and our value to the company as researchers is to optimize process parameters to optimize performance as a function of cost. This presents as an ideal opportunity to utilize tools in machine learning.

Common machine learning engineering largely requires two important pieces: 1) Build the model. 2) Build the cost function. But there is a third piece is critical to both machine learning and nanotech research: optimization. While you may have the ideal model and a becoming cost function, reconciliation of the two requires efficient optimization algorithm design if your model is going to converge in a way that doesn't waste your time (especially with the iterative nature of the data science development cycle). Gradient Descent stands as a staple of any researcher's tool kit, and for good reason: it's efficient and it's extensible.

## Theory
As an example, we'll look at the Logistic Regression model defined in "Logistic_Regression_Theory.ipynb". Say we have our LR model (linear model + sigmoid function) and our loss function as notated below...

$\large z = w^Tx + b $

$\large\hat{y} = a = \sigma(z)$ (Using $a$ for brevity)

$ \large L(\hat{y}, y) = - (y \space log\hat{y} \space + \space (1-y)\space log(1-\hat{y})$

Assuming two input variables ($w_1$, $w_2$, plus constant $b$), we can model the calculation flow of a single instance with a computation graph:

$ \large\left[\begin{array}{cc}x_1 & \\ w_1 & \\ x_2 & \\ w_2 & \\ b\end{array}\right] \implies \large [z = w_1x_1 + w_2x_2 + b] \space\to\space [\hat{y} = \sigma(z)] \space\to\space [L(a, y)]$

What we want to know is the gradient of the loss function **with respect to a given input variable**. Knowing this gradient will tell us which "direction" to tune our weight/bias parameters, as a means of moving towards the minimum of the cost function.

From here we compute the derivatives right-to-left across the computation graph, termed **backward propopgation**. Skipping over the calculus, basic chain-rule brings us to:

$\large \frac{dL}{dz} = \frac{dL}{da}\frac{da}{dz} = a - y$

$\implies \large \frac{dL}{dw_1} = x_1 \space dz = x_1(a - y)$

$\implies \large \frac{dL}{dw_2} = x_2 \space dz = x_2(a - y)$

$\implies \large \frac{dL}{db} = dz \space = a-y$

This theory summarizes calculating loss of a **single** training instance, but to minimize the cost function over an entire training set requires these computations to be done across the training set. So for every predicted value in $a$, our Cost Function can be annotated

$\large J(w, b) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)} \space log(a)^{(i)} \space + \space (1-y^{(i)})\space log(1-a^{(i)}]$

We can iteratively compute the cost and graident of given weight/bias terms, updating our parameters at each iteration, with the aim of ultimately arriving where the gradients near zero.

In the case of $J(w, b)$
* $ \large w = w - \eta \frac{\partial{J(w, b)}}{\partial{w}}$
* $ \large b = b - \eta \frac{\partial{J(w, b)}}{\partial{b}}$

with $\eta$ representing the learning rate (more on that later).

## Code 
First, we'll incorporate the helper functions built in Logistic_Regression_Theory.ipynb to facilitate calculating sigmoid of linear function, and the forward propagation between the linear function and the cost function. Based on the calculus above, we can also add functionality for *backward* propagation from the cost function to the gradients of $w$ and $b$.

In [1]:
import numpy as np

def sigmoid(x):
    """
    Helper function to compute the sigmoid of a given array
    
    input
    -----
    x: ndarray-like, output vector of linear function
    
    output
    -----
    s: ndarray-like, sigmoid of input"""
    
    sig = 1/(1+np.exp(-x))
    return sig

def init_wb(size, init_val=0):
    """
    Function to initialize weights vector/bias term 
    
    input
    -----
    size: int
        length of weights vector
    kind: int or float
        value to init 
        
    output
    -----
    w, b
        """
    if init_val == 0:
        w = np.zeros((size, 1))
        b = 0
    else:
        w = np.ones((size, 1)) * init_val
        b = init_val
    return w, b

def propagation(X, y, w, b):
    """
    Compute resulting cost function and gradients from
    forward propagation and backward propagation respectively
    
    input
    -----
    X: ndarray-like, training set to fit
    y: vector-like, X's corresponding target
    w: weight vector
    b: bias term
    
    output
    ------
    dict
        dw: numeric, gradient of the weight vector
        db: numeric, gradient of the bias vector
        cost: vector-like, cost of weight, bias terms
    """
    
    samples = X.shape[1]
    
    # Forward Propagation (left to right)
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -(1/samples) * np.sum(y * np.log(A) + (1 - y)*(np.log(1 - A)))
    
    # Backward Propagation (right to left)
    dw = (1/samples) * np.dot(X, (A - y).T)
    db = (1/samples) * np.sum(A - y)
    return {'dw': dw, 'db': db, 'cost':np.squeeze(cost)}

Now our task is to *optimize* our parameters $w$ and $b$ in a way that minimizes our cost function. If we again consider the cost function

$\large J(w, b) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)} \space log(a)^{(i)} \space + \space (1-y^{(i)})\space log(1-a^{(i)}]$

This cost function is **convex** can be modeled as a surface across dimensions $w$ and $b$. In concept, this function can be visualized as a bowl-shape. The aim of gradient descent is finding the parameters $w$ and $b$ that place converge at the bottom of this bowl; the global minimum.

In our `propagation` function, we effectively compute the gradients of our regressor parameters, coded as `dw` and `db`. To implement gradient descent, we must iteratively update our parameters at some rate $\eta$ such that their values *descend* in the direction of their steepest gradient, hopefully arriving at some global minimum. This $\eta$ is the learning rate, which controls the interval at which our parameters are changed.


This can be coded simply as...

In [103]:
def grad_descent(X, y, w, b, n, eta):
    """
    Function running gradient descent to optimize w and b
    to minimize the cost function

    input
    -----
    X: ndarray-like, training set to fit
    y: vector-like, X's corresponding target
    w: weight vector
    b: bias term
    n: number of iterations to loop
    eta: learning rate
    
    output
    ------
    
    
    """
    costs = []
    
    for i in range(n):
        
        prop = propagation(X, y, w, b)
        dw, db, cost = prop['dw'], prop['db'], prop['cost']
        
        w  = w - (eta * dw)
        b = b - (eta * db)
        
        if i % 100 == 0:
            costs.append(cost)
    
    return {'w':w, 'dw':dw, 'b':b, 'db':db, 'costs':costs}

And with our gradient descent function, we have all three pieces required in building a classifier. Our `grad_descent` function will return the optimal $w$ and $b$ for a given training set. But as our code stands right now, passing these parameters to our model will return a *probability* that an instance's target will be 1. For a classifier, we want to translate this probability to a definite outcome: 0 or 1.

In [104]:
def prediction(X, w, b):
    """
    Predict binomial classification based on trained w, b
    
    input
    -----
    w: ndarray-like
        weights vector
    b: int or float
        bias term
    X: ndarray-like
    
    output
    -----
    y_pred: np.array
        array of predictions to corresponding X instances
        """
    samples = X.shape[1]
    y_pred = np.zeros((1, samples))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        # Bin probabiliites into actual predictions
        y_prob = A[0, i]
        if y_prob > 0.5:
            y_pred[:, i] = 1
        else:
            y_pred[:, i] = 0
            
    return y_pred

And now we have all the parts of a logistic regression classifier. Look at Log_Reg_Classifier_from_Scratch.ipynb to see how we can assemble all of these parts into a fully functioning image classifier.